In [364]:
import pandas as pd
import numpy as np
import os
import xlrd
from functools import reduce
import altair as alt
alt.data_transformers.disable_max_rows()
import warnings
warnings.filterwarnings('ignore')

To read in these massive excel files first want to extract names of sheets

In [10]:
file1 = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/TenantLeasePayments_Trending_(Active)_May_2020_Sept_2020.xlsx"
file2 = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/Tenant_Lease_Payments_May 2019_Sept_2019.xlsx"

In [263]:
xls = xlrd.open_workbook(file1, on_demand=True)
may2020_sept2020 = xls.sheet_names()
may2020_sept2020 = [x for x in may2020_sept2020 if not x.endswith('Summary')] #we want to drop summary tabs 

In [271]:
# need to drop data from earlier in months where we have double data 
may2020_sept2020 = [x for x in may2020_sept2020 if not "15" in x]
may2020_sept2020 = [x for x in may2020_sept2020 if not "10" in x]


In [17]:
xls2 = xlrd.open_workbook(file2, on_demand=True)
may19_sept19 = xls2.sheet_names()

['May 31', 'June 30', 'July 31', 'August 31', 'Sept. 30']


In [192]:
def read_in(file,sheet):
    """
    inputs:
        file (str): filepath of excel file
        sheet (str): name of sheet in excel sheet
    returns:
        df (dataframe)
    Read in data from a given excel sheet, do some cleaning and then 
    make new column names that adds the date for financial transaction 
    and drops the rest of the data. 
    """
    df = pd.read_excel(file,sheet_name =sheet)
    new_header = df.iloc[4] #grab the forth row for the header
    df = df[5:] #take the data less the blank rows
    df.columns = new_header
    df = df[:-2]#drop bottom two rows, which contain summary info
    df = df.reset_index().drop(columns=["index"])
    df = df[['Tenant Lease Charge','Is Subsidized?', 'Fixed Income?', 'Tenant Rent Collected',
             'Percent Collected',"Period","Tenant"]]
    df = df.rename(columns={"Tenant Lease Charge":"Tenant Lease Charge"+" "+df["Period"][5],
                   "Tenant Rent Collected":"Tenant Rent Charge"+" "+df["Period"][5],
                  "Percent Collected":"Tenant Percent Collected"+" "+df["Period"][5],
                   'Is Subsidized?':"Is Subsidized?"+" "+df["Period"][5], 
                    'Fixed Income?':"Fixed Income?"+" "+df["Period"][5]}) 
    df = df.drop(columns={"Period"})
    return df

In [154]:
def read_in_check_status(file,sheet):
    """
    inputs:
        file (str): filepath of excel file
        sheet (str): name of sheet in excel sheet
    returns:
        df (dataframe)
    Read in data from a given excel sheet, do some cleaning and then
    create monthly columns checking subsidy status.
    """
    df = pd.read_excel(file,sheet_name =sheet)
    new_header = df.iloc[4] #grab the forth row for the header
    df = df[5:] #take the data less the blank rows
    df.columns = new_header
    df = df[:-2]#drop bottom two rows, which contain summary info
    df = df.reset_index().drop(columns=["index"])
    df = df[['Tenant', 'Is Subsidized?', 'Fixed Income?']]

    #df = df.drop(columns={"Period"})
    return df

In [310]:
def make_header(df):
    """
    inputs:
        df (dataframe)
    returns:
        df (dataframe)
    do some basic cleanup of the header row 
    """
    new_header = df.iloc[4] #grab the forth row for the header
    df = df[5:] #take the data less the blank rows
    df.columns = new_header
    return df

In [276]:
#read in the correct sheets from all files calling read_in function 
df = pd.read_excel(file2,sheet_name =may19_sept19[0])
df = make_header(df)

df_list = [df, read_in(file2,may19_sept19[1]),
           read_in(file2,may19_sept19[2]),
           read_in(file2,may19_sept19[3]),
           read_in(file2,may19_sept19[4]),
           read_in(file1,may2020_sept2020[0]),
           read_in(file1,may2020_sept2020[1]),
           read_in(file1,may2020_sept2020[2]),
           read_in(file1,may2020_sept2020[3]),
           read_in(file1,may2020_sept2020[4])]

In [278]:
#This lambda function then merges all the dataframes from df_list together 
df_final = reduce(lambda left,right: pd.merge(left,right,on='Tenant'), df_list)

df_final = df_final.drop(columns=["Name","Period"]) #drop cols

In [281]:
#write out initial data to csv
df_final.to_csv("Inital_eden_rent_roll_data_19-20_v2.csv")

Q: “Have rent collections changed in 2020 compared to last year (e.g., collective rent roll shortfalls, number of tenants with underpayment or no payment in each month)?”
 


Need to melt dataset so each row is amount of rent collected in a given month to be able to do groupbys

In [212]:
#grab initial fixed columns for first month in dataset 
fixed_cols = ['Property', 'Property Name', 'City', 'County', 'Unit', 'Tenant',
       'Is Subsidized?', 'Fixed Income?']

In [286]:
# additionally grab just monthly percentage payment columns 
percent_collected = [num for num in df_final.columns if ("Percent") in num]

In [291]:
#merge these two lists in the total list of columns we want so we can melt dataset 
# for this initial analysis assuming that fixed income status and stuff do not change
collect = fixed_cols + percent_collected
df_rent_collected = df_final[collect]

In [292]:
#create a melted dataset so that each row is a payment 
monthly_percent_collected = pd.melt(df_rent_collected, id_vars=fixed_cols,var_name='Percent Collected')

In [293]:
# there are 90,640 payments in this dataset 
len(monthly_percent_collected)

90640

In [295]:
# to group by year, need a year col 
monthly_percent_collected["year"] = monthly_percent_collected["Percent Collected"].str[-2:]

In [301]:
len(monthly_percent_collected[monthly_percent_collected.value > 100])

14577

In [308]:
#16 percent of all payments exceed total, implying paying back rent. 
len(monthly_percent_collected[monthly_percent_collected.value > 100])/ len(monthly_percent_collected)

0.16082303618711385

In [302]:
#negative rent total 
len(monthly_percent_collected[monthly_percent_collected.value < 0])

63

In [309]:
(len(monthly_percent_collected[monthly_percent_collected.value < 0])/len(monthly_percent_collected))

0.0006950573698146514

In [323]:
#I'm going to drop these negative values as I believe them to be errors. 
monthly_percent_collected = monthly_percent_collected[monthly_percent_collected.value >= 0]

In [327]:
monthly_percent_collected.value = monthly_percent_collected.value.astype(float) #make value type that works with groupby 
monthly_percent_collected[monthly_percent_collected.value <=100][["year","value"]].groupby("year").describe()

/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


value                                                         
        count       mean        std  min      25%    50%    75%    max
year                                                                  
19    37847.0  78.487889  40.651772  0.0   98.735  100.0  100.0  100.0
20    38153.0  91.361198  26.732037  0.0  100.000  100.0  100.0  100.0

In [329]:
#count the number of non payments per year 
monthly_percent_collected[["year","value"]].groupby('year').agg(lambda x: x.eq(0).sum())

,value
year,
19,7898.0
20,2727.0


In [331]:
7898.0 + 2727.0

10625.0

Q: “For tenants who are short or don't pay rent...is that typically a one-off thing or are they likely to be in arrears multiple months. How far short do they tend to be? Has that shifted in 2020?”

In [338]:
#create a datetime column to make altair happy 
monthly_percent_collected_chart = monthly_percent_collected
monthly_percent_collected_chart["Percent Collected"] = monthly_percent_collected_chart["Percent Collected"].str[-8:]
monthly_percent_collected_chart["Percent Collected"] = pd.to_datetime(monthly_percent_collected_chart["Percent Collected"])

In [474]:
monthly_percent_collected_chart = monthly_percent_collected_chart[["value","Percent Collected"]].groupby("Percent Collected").mean().reset_index()

NOTE I DROPPED THE CONFIDENCE BANDS FROM THIS CHART TO MAKE THE NOTEBOOK FILE SMALL ENOUGH TO LOAD ONTO GITHUB
CAN BRING THEM BACK EASILY IF NEEDED 

In [475]:
line = line_chart = alt.Chart(monthly_percent_collected_chart).mark_line(color="#FFAA00").encode(
        alt.X('Percent Collected',
        scale=alt.Scale(zero=False)
    ),


    alt.Y("value", scale=alt.Scale(zero=False))
)

band = line_chart = alt.Chart(monthly_percent_collected_chart).mark_errorband(extent='ci').encode(
        alt.X('Percent Collected',
        scale=alt.Scale(zero=False)
    ),


    alt.Y("value", scale=alt.Scale(zero=False))
)

band + line

alt.LayerChart(...)

In [371]:
df_rent_collected

4,Property,Property Name,City,County,Unit,Tenant,Is Subsidized?,Fixed Income?,Tenant Percent Collected 05/01/19 - 05/31/19,Tenant Percent Collected 06/01/19 - 06/30/19,Tenant Percent Collected 07/01/19 - 07/31/19,Tenant Percent Collected 08/01/19 - 08/31/19,Tenant Percent Collected 09/01/19 - 09/30/19,Tenant Percent Collected 05/01/20 - 05/31/20,Tenant Percent Collected 06/01/20 - 06/30/20,Tenant Percent Collected 07/01/20 - 07/31/20,Tenant Percent Collected 08/01/20 - 08/31/20,Tenant Percent Collected 09/01/20 - 09/30/20
0,304,Almond Terrace,Manteca,San Joaquin,A101,t0028148,Yes,NaN,98.98,100,100,100,100,100,100,100,100,100
1,304,Almond Terrace,Manteca,San Joaquin,A102,t0009751,Yes,NaN,100,104.37,95.63,100.87,100,100.44,99.56,100,100,101.75
2,304,Almond Terrace,Manteca,San Joaquin,A103,t0029469,NaN,NaN,0,0,0,0,0,100,100,100,100,100
3,304,Almond Terrace,Manteca,San Joaquin,A104,t0023101,Yes,NaN,100,100,100,100,100,100,100,100,100,100
4,304,Almond Terrace,Manteca,San Joaquin,A105,t0000702,Yes,SS,100,100,100,100,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9059,998,Tennyson Garden Apartments,Hayward,Alameda,983-303,t0028619,Yes,SS,101.34,100,100.53,100,100,101.13,101.13,101.13,107.01,104.52
9060,998,Tennyson Garden Apartments,Hayward,Alameda,983-304,t0028620,Yes,NaN,140.04,140.41,139.66,139.66,116.46,0,100,200,100,52.5
9061,998,Tennyson Garden Apartments,Hayward,Alameda,983-305,t0028622,Yes,NaN,0,0,0,0,100,100.5,100.5,100.5,100.5,100
9062,998,Tennyson Garden Apartments,Hayward,Alameda,983-306,t0028624,NaN,NaN,100,100,100,100,100,100,100,100,100,100


In [379]:
#melt, group by person, do a value counts 
people_with_nonpayment = df_rent_collected.loc[~(df_rent_collected != 0).all(axis=1), :]
len(people_with_nonpayment)

2647

In [412]:
nonpayment_only = people_with_nonpayment[['Tenant Percent Collected 05/01/19 - 05/31/19',
       'Tenant Percent Collected 06/01/19 - 06/30/19',
       'Tenant Percent Collected 07/01/19 - 07/31/19',
       'Tenant Percent Collected 08/01/19 - 08/31/19',
       'Tenant Percent Collected 09/01/19 - 09/30/19',
       'Tenant Percent Collected 05/01/20 - 05/31/20',
       'Tenant Percent Collected 06/01/20 - 06/30/20',
       'Tenant Percent Collected 07/01/20 - 07/31/20',
       'Tenant Percent Collected 08/01/20 - 08/31/20',
       'Tenant Percent Collected 09/01/20 - 09/30/20']]

In [408]:
people_with_nonpayment.columns

Index(['Property', 'Property Name', 'City', 'County', 'Unit', 'Tenant',
       'Is Subsidized?', 'Fixed Income?',
       'Tenant Percent Collected 05/01/19 - 05/31/19',
       'Tenant Percent Collected 06/01/19 - 06/30/19',
       'Tenant Percent Collected 07/01/19 - 07/31/19',
       'Tenant Percent Collected 08/01/19 - 08/31/19',
       'Tenant Percent Collected 09/01/19 - 09/30/19',
       'Tenant Percent Collected 05/01/20 - 05/31/20',
       'Tenant Percent Collected 06/01/20 - 06/30/20',
       'Tenant Percent Collected 07/01/20 - 07/31/20',
       'Tenant Percent Collected 08/01/20 - 08/31/20',
       'Tenant Percent Collected 09/01/20 - 09/30/20'],
      dtype='object', name=4)

In [400]:
melted_nonpayment = pd.melt(people_with_nonpayment, id_vars=fixed_cols,var_name='Percent Collected')

In [401]:
melted_nonpayment = melted_nonpayment[['Tenant', 'value']]
melted_nonpayment.value = melted_nonpayment.value.astype(float)

In [405]:
melted_nonpayment.head()

,Tenant,value
0,t0029469,0.0
1,t0000690,0.0
2,t0098684,0.0
3,t0852179,0.0
4,t0029300,0.0


In [ ]:
(melted_nonpayment.value == 0).sum(axis=1)

In [398]:
melted_nonpayment_des = melted_nonpayment.groupby("Tenant").describe().reset_index()

In [396]:
melted_nonpayment.head()

Tenant value                                                         
            count    mean        std  min    25%     50%       75%     max
0  apj00041  10.0   0.000   0.000000  0.0   0.00    0.00    0.0000    0.00
1  apj00079  10.0  61.500  53.094951  0.0   0.00  100.00  100.0000  113.89
2     fga11  10.0  96.427  34.968254  0.0  97.32  111.23  113.9575  114.06
3     fga17  10.0  98.121  46.665885  0.0  97.27   97.27   98.9700  197.95
4     pga22  10.0  70.000  48.304589  0.0  25.00  100.00  100.0000  100.00

In [421]:
monthly_percent_collected["Fixed Income?"] = monthly_percent_collected["Fixed Income?"].replace(np.nan, 'No Assistance', regex=True)
monthly_percent_collected[["Fixed Income?","value"]].groupby("Fixed Income?").mean()

,value
Fixed Income?,
No Assistance,89.589170
SS,99.160790
SS/SSI,100.767098
SSI,100.319768


In [445]:
monthly_percent_collected.head()

,Property,Property Name,City,County,Unit,Tenant,Is Subsidized?,Fixed Income?,Percent Collected,value,year
0,304,Almond Terrace,Manteca,San Joaquin,A101,t0028148,Yes,No Assistance,2019-05-31,98.98,19
1,304,Almond Terrace,Manteca,San Joaquin,A102,t0009751,Yes,No Assistance,2019-05-31,100.00,19
2,304,Almond Terrace,Manteca,San Joaquin,A103,t0029469,NaN,No Assistance,2019-05-31,0.00,19
3,304,Almond Terrace,Manteca,San Joaquin,A104,t0023101,Yes,No Assistance,2019-05-31,100.00,19
4,304,Almond Terrace,Manteca,San Joaquin,A105,t0000702,Yes,SS,2019-05-31,100.00,19


In [428]:
df_rent_collected["Fixed Income?"] = df_rent_collected["Fixed Income?"].replace(np.nan, 'No Assistance', regex=True)
df_rent_collected["Is Subsidized?"] = df_rent_collected["Is Subsidized?"].replace(np.nan, 'No', regex=True)
df_rent_collected["Is Subsidized?"] = df_rent_collected["Is Subsidized?"] + " " + "Subsidy"
df_rent_collected["Assistance"] = df_rent_collected["Is Subsidized?"] +" "+ df_rent_collected["Fixed Income?"]

In [429]:
df_rent_collected["Fixed Income?"].value_counts()

No Assistance    6239
SS               1413
SSI               768
SS/SSI            644
Name: Fixed Income?, dtype: int64

In [440]:
df_rent_collected["Is Subsidized?"].value_counts()

No Subsidy     5617
Yes Subsidy    3447
Name: Is Subsidized?, dtype: int64

In [443]:
4507 /(5617 + 3447)

0.49724183583406883

In [438]:
df_rent_collected["Assistance"].value_counts()

No Subsidy No Assistance     4507
Yes Subsidy No Assistance    1732
Yes Subsidy SS                768
No Subsidy SS                 645
Yes Subsidy SSI               485
Yes Subsidy SS/SSI            462
No Subsidy SSI                283
No Subsidy SS/SSI             182
Name: Assistance, dtype: int64

In [446]:
df_rent_collected.head()

4,Property,Property Name,City,County,Unit,Tenant,Is Subsidized?,Fixed Income?,Tenant Percent Collected 05/01/19 - 05/31/19,Tenant Percent Collected 06/01/19 - 06/30/19,Tenant Percent Collected 07/01/19 - 07/31/19,Tenant Percent Collected 08/01/19 - 08/31/19,Tenant Percent Collected 09/01/19 - 09/30/19,Tenant Percent Collected 05/01/20 - 05/31/20,Tenant Percent Collected 06/01/20 - 06/30/20,Tenant Percent Collected 07/01/20 - 07/31/20,Tenant Percent Collected 08/01/20 - 08/31/20,Tenant Percent Collected 09/01/20 - 09/30/20,Assistance
0,304,Almond Terrace,Manteca,San Joaquin,A101,t0028148,Yes Subsidy,No Assistance,98.98,100,100,100,100,100,100,100,100,100,Yes Subsidy No Assistance
1,304,Almond Terrace,Manteca,San Joaquin,A102,t0009751,Yes Subsidy,No Assistance,100,104.37,95.63,100.87,100,100.44,99.56,100,100,101.75,Yes Subsidy No Assistance
2,304,Almond Terrace,Manteca,San Joaquin,A103,t0029469,No Subsidy,No Assistance,0,0,0,0,0,100,100,100,100,100,No Subsidy No Assistance
3,304,Almond Terrace,Manteca,San Joaquin,A104,t0023101,Yes Subsidy,No Assistance,100,100,100,100,100,100,100,100,100,100,Yes Subsidy No Assistance
4,304,Almond Terrace,Manteca,San Joaquin,A105,t0000702,Yes Subsidy,SS,100,100,100,100,100,100,100,100,100,100,Yes Subsidy SS


In [448]:
monthly_percent_collected["Assistance"] = monthly_percent_collected["Is Subsidized?"] +" "+ monthly_percent_collected["Fixed Income?"]

In [470]:
monthly_percent_collected[["Fixed Income?","Percentage of Rent Paid","year"]].groupby(["Fixed Income?","year"]).mean()

Percentage of Rent Paid
Fixed Income? year                         
No Assistance 19                  82.987580
              20                  96.193513
SS            19                  98.357691
              20                  99.964116
SS/SSI        19                 101.353494
              20                 100.180702
SSI           19                 101.084756
              20                  99.554780

In [471]:
monthly_percent_collected[["Is Subsidized?","Percentage of Rent Paid","year"]].groupby(["Is Subsidized?","year"]).mean()

Percentage of Rent Paid
Is Subsidized? year                         
No Subsidy     19                  77.527213
               20                  95.897760
Yes Subsidy    19                 105.675739
               20                  99.720503

In [469]:
monthly_percent_collected.head()

,Property,Property Name,City,County,Unit,Tenant,Is Subsidized?,Fixed Income?,Percent Collected,Percentage of Rent Paid,year,Assistance,Aid
0,304,Almond Terrace,Manteca,San Joaquin,A101,t0028148,Yes Subsidy,No Assistance,2019-05-31,98.98,19,Yes Subsidy No Assistance,Aid
1,304,Almond Terrace,Manteca,San Joaquin,A102,t0009751,Yes Subsidy,No Assistance,2019-05-31,100.00,19,Yes Subsidy No Assistance,Aid
2,304,Almond Terrace,Manteca,San Joaquin,A103,t0029469,No Subsidy,No Assistance,2019-05-31,0.00,19,No Subsidy No Assistance,No Aid
3,304,Almond Terrace,Manteca,San Joaquin,A104,t0023101,Yes Subsidy,No Assistance,2019-05-31,100.00,19,Yes Subsidy No Assistance,Aid
4,304,Almond Terrace,Manteca,San Joaquin,A105,t0000702,Yes Subsidy,SS,2019-05-31,100.00,19,Yes Subsidy SS,Aid


In [450]:
monthly_percent_collected["Fixed Income?"] = monthly_percent_collected["Fixed Income?"].replace(np.nan, 'No Assistance', regex=True)
monthly_percent_collected["Is Subsidized?"] = monthly_percent_collected["Is Subsidized?"].replace(np.nan, 'No', regex=True)
monthly_percent_collected["Is Subsidized?"] = monthly_percent_collected["Is Subsidized?"] + " " + "Subsidy"
monthly_percent_collected["Assistance"] = monthly_percent_collected["Is Subsidized?"] +" "+ monthly_percent_collected["Fixed Income?"]

In [451]:
monthly_percent_collected["Assistance"].value_counts()

No Subsidy No Assistance     45062
Yes Subsidy No Assistance    17271
Yes Subsidy SS                7677
No Subsidy SS                 6449
Yes Subsidy SSI               4848
Yes Subsidy SS/SSI            4620
No Subsidy SSI                2830
No Subsidy SS/SSI             1820
Name: Assistance, dtype: int64

In [458]:
monthly_percent_collected.head()

,Property,Property Name,City,County,Unit,Tenant,Is Subsidized?,Fixed Income?,Percent Collected,value,year,Assistance
0,304,Almond Terrace,Manteca,San Joaquin,A101,t0028148,Yes Subsidy,No Assistance,2019-05-31,98.98,19,Yes Subsidy No Assistance
1,304,Almond Terrace,Manteca,San Joaquin,A102,t0009751,Yes Subsidy,No Assistance,2019-05-31,100.00,19,Yes Subsidy No Assistance
2,304,Almond Terrace,Manteca,San Joaquin,A103,t0029469,No Subsidy,No Assistance,2019-05-31,0.00,19,No Subsidy No Assistance
3,304,Almond Terrace,Manteca,San Joaquin,A104,t0023101,Yes Subsidy,No Assistance,2019-05-31,100.00,19,Yes Subsidy No Assistance
4,304,Almond Terrace,Manteca,San Joaquin,A105,t0000702,Yes Subsidy,SS,2019-05-31,100.00,19,Yes Subsidy SS


In [459]:
def any_aid(x):
    if x == "No Subsidy No Assistance":
        return "No Aid"
    else:
        return "Aid"

In [461]:
monthly_percent_collected["Aid"] = monthly_percent_collected.Assistance.apply(lambda x: any_aid(x))

In [467]:
monthly_percent_collected = monthly_percent_collected.rename(columns={"value":"Percentage of Rent Paid"})

In [468]:
monthly_percent_collected[["Aid","Percentage of Rent Paid","year"]].groupby(["Aid","year"]).mean()

Percentage of Rent Paid
Aid    year                         
Aid    19                 102.643698
       20                  99.771580
No Aid 19                  73.657068
       20                  94.904618

Q: How do rent shortfalls vary across presence/type of subsidy, and has that shifted in 2020? (Also, did you determine the extent to which subsidy status varies over time, if at all?)

In [ ]:
--
--Are rent shortfalls concentrated in certain properties or regions of the state?
	These are both just groupby questions - like just groupby icty county/and fix income type 
